In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [2]:
torch.version.cuda

'11.6'

In [3]:
dataset = Planetoid(root='tutorial1', name='Cora')

Visualizing some properties of Planetoid dataset

In [4]:
#General information about the dataset
print(dataset.data)

print(f'Number of entire graphs: {len(dataset)}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Node feature dimension: {dataset.num_node_features}')
print(f'Edge feature dimension: {dataset.num_edge_features}')

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of entire graphs: 1
Number of classes: 7
Node feature dimension: 1433
Edge feature dimension: 0


In [5]:
#Getting the data from dataset
data = dataset[0]

## Defining GraphSage

In [28]:
class GraphSage(torch.nn.Module):
    def __init__(self):
        super(GraphSage, self).__init__()
        
        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return x

### Training

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GraphSage().to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [30]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model()
    loss = criterion(output[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [31]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.4580, Test: 0.4960
Epoch: 020, Val: 0.6240, Test: 0.6410
Epoch: 030, Val: 0.6560, Test: 0.6850
Epoch: 040, Val: 0.6800, Test: 0.7070
Epoch: 050, Val: 0.6820, Test: 0.7120
Epoch: 060, Val: 0.6820, Test: 0.7120
Epoch: 070, Val: 0.6820, Test: 0.7120
Epoch: 080, Val: 0.6820, Test: 0.7120
Epoch: 090, Val: 0.6820, Test: 0.7120
